In [6]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://en.wikipedia.org/api/rest_v1/page/html/math')

with open('test.html', 'wb') as f:
    f.write(r.content)

soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html about="https://en.wikipedia.org/wiki/Special:Redirect/revision/1174909284" prefix="dc: http://purl.org/dc/terms/ mw: http://mediawiki.org/rdf/">
 <head prefix="mwr: https://en.wikipedia.org/wiki/Special:Redirect/">
  <meta content="497b04e0-50ac-11ee-8b55-b7d6f4df28da" property="mw:TimeUuid"/>
  <meta charset="utf-8"/>
  <meta content="18831" property="mw:pageId"/>
  <meta content="0" property="mw:pageNamespace"/>
  <link rel="dc:replaces" resource="mwr:revision/1174909097"/>
  <meta content="49cf5b79b5d3833f7dff71b8091c92195f9fa30b" property="mw:revisionSHA1"/>
  <meta content="2023-09-11T14:05:25.000Z" property="dc:modified"/>
  <meta content="2.8.0" property="mw:htmlVersion"/>
  <meta content="2.8.0" property="mw:html:version"/>
  <link href="//en.wikipedia.org/wiki/Mathematics" rel="dc:isVersionOf"/>
  <base href="//en.wikipedia.org/wiki/"/>
  <title>
   Mathematics
  </title>
  <meta content="ext.cite.ux-enhancements" property="mw:generalModules"/>
  <meta co

In [18]:
anchors = []
links = []
topics = set()
for a in soup.select('a[rel^="mw:WikiLink"]'):
    anchors.append(a)
    topics.append(str(a.string))
    links.push(a.get('href'))

#for t in topics[:int(len(topics)/4)]:
for t in topics:

    print(t)

print(len(topics))


AttributeError: 'set' object has no attribute 'append'